In [46]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

#import packages
import pandas as pd
import numpy as np
import datetime as dt

In [47]:
# Import data
data_crime = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/crime_data_edited.csv')
data_share = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/election_data_edited.csv')

In [48]:
data_share.dropna(thresh=4, inplace=True)
data_share.drop_duplicates(subset=['plz', 'city', 'party', 'state', 'city_id', 'year'], inplace=True)

In [49]:
# get into datetime format
data_crime['year'] = data_crime['date'].astype(str).str.split('.', expand=True)[2]
data_crime['month'] = data_crime['date'].astype(str).str.split('.', expand=True)[1]
data_crime['day'] = data_crime['date'].astype(str).str.split('.', expand=True)[0]
data_crime['date'] = pd.to_datetime(data_crime[['year', 'month', 'day']].astype(float), errors='ignore')

In [50]:
# Merge crime and candidate statistics with merge over city string
data_crime = data_crime[(data_crime['Land'] == 14) | (data_crime['Land'] == 8) | (data_crime['Land'] == 16) | (data_crime['Land'] == 12)]
data_share['city'] = data_share['city'].str.replace(r'Stadt ', '')
data_share['city'] = data_share['city'].str.replace('Ä', 'A').str.replace('Ö', 'O').str.replace('Ü', 'U')

list_cities_share = data_share.sort_values('city')['city'].unique()
list_cities_crime = data_crime['city'].unique()

data_crime.rename({'Land':'state'}, axis=1, inplace=True)
data_crime['state'] = data_crime['state'].map({16:'TH', 8:'BW', 14:'SN', 12:'BB'})

In [51]:
data_share.rename({'year':'cycle_1'}, axis=1, inplace=True)
data_share['cycle_2'] = data_share['cycle_1']
data_share['cycle_3'] = data_share['cycle_1']

Merging over years and cities? How many different cutoffs? Directly on election day, one month before and three months before

Cut-off days for the three states: 05.26.2019, 04.26.2019, 02.26.2019

In [52]:
# creating different cycle variables in crime data to merge over
#
data_crime['cycle_1'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 5, 26, 0, 0), 'cycle_1'] = 2019

data_crime['cycle_2'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 4, 26, 0, 0), 'cycle_2'] = 2019

data_crime['cycle_3'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 2, 26, 0, 0), 'cycle_3'] = 2019

In [53]:
# remove duplicates from data_share
# The following mistake happened: during merger full line and line w/out merger success being kept
# If done before, file will not contain unmatched entities -> necessary for manual inspection
#data_share.drop_duplicates(subset=['city','party','Anzahl Bewerber','darunter Frauen','cycle_1','state'], ignore_index=True, inplace=True)

In [54]:
# Find double cities in data share
# we cannot use plz here, since crime data doesn't have any of that -> merger on plz doesnt work
# list_double = pd.DataFrame()
# for city in data_share['city'].unique():
    
#     data_city = data_share[data_share['city'] == city]
#     for party in data_city['party'].unique():
        
#         data_party = data_city[data_city['party'] == party]
#         for election in data_party['cycle_1'].unique():
            
#             data_election = data_party[data_party['cycle_1'] == election]
            
#             if len(data_election) > 1:
                
#                 # append to dataset of 
#                 list_double = list_double.append(data_election)
                

In [55]:
# drop cities that are not uniquely identifable by cityname
#list_double_cities = data_share[data_share.city.isin(list_double.city)]['city'].unique()
#list_to_drop_crime = data_crime[data_crime.city.isin(list_double.city)]
#data_crime = data_crime[~data_crime.city.isin(list_to_drop_crime.city)]

In [56]:
#data_share.dropna(thresh, inplace=True)

# Generate crime_count variable for easier handling
data_crime['crime_count'] = 1
data_crime.drop('Unnamed: 0', axis=1, inplace=True)
data_crime.dropna(subset=['party'], inplace=True)

In [57]:
data_merged = pd.merge(data_crime, data_share, on=['plz', 'city', 'cycle_1', 'party', 'state', 'cycle_2', 'cycle_3', 'city_id'], how='right', validate='m:1')
data_merged.drop(['Unnamed: 0'], axis=1, inplace=True)
data_merged['crime_count'] = data_merged['crime_count'].fillna(0)
data_merged

,crime,city,law,date,background,suspects,party,state,plz,city_id,...,cycle_1,cycle_2,cycle_3,crime_count,Anzahl Bewerber,Prozent Frauen,darunter Frauen,Land,party_long,city_id_bb
0,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,2019.0,2019.0,2019.0,1.0,60.0,26.666667,16.0,8.0,NaN,NaN
1,Verleumdung ohne Verleumdung ohne sexuelle Gru...,Stuttgart,STGB-187,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,2019.0,2019.0,2019.0,1.0,60.0,26.666667,16.0,8.0,NaN,NaN
2,Volksverhetzung,Stuttgart,STGB-130,2019-08-12,Rechts,1.0,AfD,BW,70173.0,8111000.0,...,2019.0,2019.0,2019.0,1.0,60.0,26.666667,16.0,8.0,NaN,NaN
3,Sachbeschadigung,Stuttgart,STGB-303,2019-08-26,Links,0.0,AfD,BW,70173.0,8111000.0,...,2019.0,2019.0,2019.0,1.0,60.0,26.666667,16.0,8.0,NaN,NaN
4,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,2019.0,2019.0,2019.0,1.0,60.0,26.666667,16.0,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21387,NaN,Zossen,NaN,NaT,NaN,NaN,GRÜNE,BB,15806.0,12072477.0,...,2019.0,2019.0,2019.0,0.0,5.0,NaN,1.0,NaN,BÜNDNIS 90/DIE GRÜNEN,72477.0
21388,NaN,Zossen,NaN,NaT,NaN,NaN,FDP,BB,15806.0,12072477.0,...,2019.0,2019.0,2019.0,0.0,1.0,NaN,1.0,NaN,Freie Demokratische Partei,72477.0
21389,NaN,Uckerfelde,NaN,NaT,NaN,NaN,AfD,BB,17291.0,12073578.0,...,2019.0,2019.0,2019.0,0.0,1.0,NaN,0.0,NaN,Alternative für Deutschland,73578.0
21390,NaN,Zichow,NaN,NaT,NaN,NaN,CDU,BB,16306.0,12073645.0,...,2019.0,2019.0,2019.0,0.0,5.0,NaN,1.0,NaN,Christlich Demokratische Union Deutschlands,73645.0


In [58]:
data_merged_2 = data_merged.groupby('city').count()
print(f"Total num of cities w/ crime & target: {len(data_crime['city'].unique())}, \n")
print(f"Total num of cities w/ election: {len(data_share['city'].unique())}, \n")
print(f"Total num of cities w/ crime & election: {len(data_merged_2[data_merged_2['crime'] != 0])}, \n")
print(f"Total num of crime in df: {len(data_merged[~data_merged['crime'].isna()])}")

Total num of cities w/ crime & target: 486, 

Total num of cities w/ election: 2334, 

Total num of cities w/ crime & election: 237, 

Total num of crime in df: 2958


### Generate various measures of crime

In [59]:
# regular summarization across cities and parties
data_crime_base = data_crime.groupby(['city', 'background', 'party', 'cycle_1', 'state'], as_index=False).sum()
data_crime_party = data_crime.groupby(['city', 'party', 'cycle_1', 'state'], as_index=False).sum()
data_crime_lr = data_crime[(data_crime['background'] == 'Links') | (data_crime['background'] == 'Rechts') | (data_crime['background'].isna())] #drop unidentified crime for left-right analysis
data_crime_lr_merge= data_crime_lr.groupby(['city', 'background', 'cycle_1', 'state'], as_index=False).sum()
data_crime_city = data_crime.groupby(['city', 'cycle_1', 'state'], as_index=False).sum()
data_crime_base.rename({'crime_count':'crime_count_party_lr'}, axis=1, inplace=True)
data_crime_party.rename({'crime_count':'crime_count_party'}, axis=1, inplace=True)
data_crime_lr_merge.rename({'crime_count':'crime_count_lr'}, axis=1, inplace=True)
data_crime_city.rename({'crime_count':'crime_count_city'}, axis=1, inplace=True)

In [60]:
# putting all measures into on dataframe. CAUTION: Alsways do drop_duplicates and dropna if other than crime_count used
data_merged = pd.merge(data_merged, data_crime_base[['city', 'background', 'party', 'cycle_1', 'state', 'crime_count_party_lr']], on=['city', 'background', 'party', 'cycle_1', 'state'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_crime_party[['city', 'party', 'cycle_1', 'state', 'crime_count_party']], on=['city', 'party', 'cycle_1', 'state'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_crime_lr_merge[['city', 'background', 'cycle_1', 'state', 'crime_count_lr']], on=['city', 'background', 'cycle_1', 'state'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_crime_city[['city', 'cycle_1', 'state', 'crime_count_city']], on=['city', 'cycle_1', 'state'], how='left', validate='m:1') 

In [61]:
for col in ('crime_count_party_lr','crime_count_party','crime_count_lr','crime_count_city'):
    data_merged[col].fillna(0, inplace=True)
data_merged.tail(5)

,crime,city,law,date,background,suspects,party,state,plz,city_id,...,Anzahl Bewerber,Prozent Frauen,darunter Frauen,Land,party_long,city_id_bb,crime_count_party_lr,crime_count_party,crime_count_lr,crime_count_city
21387,NaN,Zossen,NaN,NaT,NaN,NaN,GRÜNE,BB,15806.0,12072477.0,...,5.0,NaN,1.0,NaN,BÜNDNIS 90/DIE GRÜNEN,72477.0,0.0,0.0,0.0,0.0
21388,NaN,Zossen,NaN,NaT,NaN,NaN,FDP,BB,15806.0,12072477.0,...,1.0,NaN,1.0,NaN,Freie Demokratische Partei,72477.0,0.0,0.0,0.0,0.0
21389,NaN,Uckerfelde,NaN,NaT,NaN,NaN,AfD,BB,17291.0,12073578.0,...,1.0,NaN,0.0,NaN,Alternative für Deutschland,73578.0,0.0,0.0,0.0,0.0
21390,NaN,Zichow,NaN,NaT,NaN,NaN,CDU,BB,16306.0,12073645.0,...,5.0,NaN,1.0,NaN,Christlich Demokratische Union Deutschlands,73645.0,0.0,0.0,0.0,0.0
21391,NaN,Schoneberg,NaN,NaT,NaN,NaN,DIE LINKE,BB,16278.0,12073505.0,...,1.0,NaN,0.0,NaN,DIE LINKE,73505.0,0.0,0.0,0.0,0.0


In [62]:
data_merged['party_lr'] = 'right'
data_merged.loc[(data_merged['party'] == 'SPD') | (data_merged['party'] == 'GRÜNE') | (data_merged['party'] == 'DIE LINKE'), 'party_lr'] = 'left' 

In [63]:
data_share['state'].unique()

array(['BW', 'SN', 'TH', 'BB'], dtype=object)

### Generate aggregation of fem_share

In [64]:
data_elec_by_crime_background = data_merged.groupby(['background', 'city', 'cycle_1', 'state', 'plz'], as_index=False).sum()[['background', 'city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
data_elec_by_crime_background.rename({'Anzahl Bewerber':'bewerber_hit_by_lr', 'darunter Frauen':'fem_hit_by_lr'}, axis=1, inplace=True)

city_crime = data_merged.groupby(['city', 'cycle_1', 'state', 'plz'], as_index=False).sum()[['city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
city_crime.rename({'Anzahl Bewerber':'Bewerber_city', 'darunter Frauen':'Frauen_city'}, axis=1, inplace=True)

data_elec_by_crime_background_party = data_merged.groupby(['background', 'city', 'cycle_1', 'state', 'plz', 'party'], as_index=False).sum()[['background', 'party', 'city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
data_elec_by_crime_background_party.rename({'Anzahl Bewerber':'bewerber_party_hit_by_lr', 'darunter Frauen':'fem_party_hit_by_lr'}, axis=1, inplace=True)


In [65]:
data_merged = pd.merge(data_merged, city_crime, on=['city', 'cycle_1', 'state', 'plz'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_elec_by_crime_background, on=['background', 'city', 'cycle_1', 'state', 'plz'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_elec_by_crime_background_party, on=['background', 'city', 'cycle_1', 'state', 'plz', 'party'], how='left', validate='m:1')

In [66]:
for col in ('bewerber_hit_by_lr','fem_hit_by_lr','Bewerber_city','Frauen_city', 'bewerber_party_hit_by_lr', 'fem_party_hit_by_lr'):
    data_merged[col].fillna(0, inplace=True)
data_merged

,crime,city,law,date,background,suspects,party,state,plz,city_id,...,crime_count_party,crime_count_lr,crime_count_city,party_lr,Bewerber_city,Frauen_city,bewerber_hit_by_lr,fem_hit_by_lr,bewerber_party_hit_by_lr,fem_party_hit_by_lr
0,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,35.0,21.0,109.0,right,4560.0,1722.0,1260.0,410.0,900.0,240.0
1,Verleumdung ohne Verleumdung ohne sexuelle Gru...,Stuttgart,STGB-187,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,35.0,21.0,109.0,right,4560.0,1722.0,1260.0,410.0,900.0,240.0
2,Volksverhetzung,Stuttgart,STGB-130,2019-08-12,Rechts,1.0,AfD,BW,70173.0,8111000.0,...,35.0,16.0,109.0,right,4560.0,1722.0,600.0,257.0,120.0,32.0
3,Sachbeschadigung,Stuttgart,STGB-303,2019-08-26,Links,0.0,AfD,BW,70173.0,8111000.0,...,35.0,21.0,109.0,right,4560.0,1722.0,1260.0,410.0,900.0,240.0
4,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,35.0,21.0,109.0,right,4560.0,1722.0,1260.0,410.0,900.0,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21387,NaN,Zossen,NaN,NaT,NaN,NaN,GRÜNE,BB,15806.0,12072477.0,...,0.0,0.0,0.0,left,40.0,8.0,0.0,0.0,0.0,0.0
21388,NaN,Zossen,NaN,NaT,NaN,NaN,FDP,BB,15806.0,12072477.0,...,0.0,0.0,0.0,right,40.0,8.0,0.0,0.0,0.0,0.0
21389,NaN,Uckerfelde,NaN,NaT,NaN,NaN,AfD,BB,17291.0,12073578.0,...,0.0,0.0,0.0,right,1.0,0.0,0.0,0.0,0.0,0.0
21390,NaN,Zichow,NaN,NaT,NaN,NaN,CDU,BB,16306.0,12073645.0,...,0.0,0.0,0.0,right,5.0,1.0,0.0,0.0,0.0,0.0


In [67]:
data_merged['state'].unique()

array(['BW', 'SN', 'TH', 'BB'], dtype=object)

In [68]:
# p_female works with crime_count_party_lr and crime_count_party
data_merged['p_female'] = data_merged['darunter Frauen']/data_merged['Anzahl Bewerber']
data_merged['p_female_lr'] = data_merged['fem_hit_by_lr']/data_merged['bewerber_hit_by_lr']
data_merged['p_female_city'] = data_merged['Frauen_city']/data_merged['Bewerber_city']
data_merged['p_female_party_lr'] = data_merged['fem_party_hit_by_lr']/data_merged['bewerber_party_hit_by_lr']

### What we have now:
* Panel data set in levels -> always drop duplicates depending on level of analysis

#### Later on, insert control variables here

### What we need:
* Difference in election shares (caution still holds)

In [69]:
## generate differences in party shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'party', 'p_female'])
data_panel = pd.DataFrame()
for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]

        for party in data_city['party'].unique():
            data_party = data_city[data_city['party'] == party]

            # drop units with only one party observation
            if len(data_party) > 1:

                diff = data_party[(data_party['cycle_1'] == 2019)]['p_female'].values - data_party[(data_party['cycle_1'] == 2014)]['p_female'].values
                data_merged.loc[(data_merged['party'] == party) & (data_merged['city'] == city), 'p_female_diff'] = diff[0]

In [70]:
# generate differences in city shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'p_female_city'])
data_panel = pd.DataFrame()

for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]

        if len(data_city) > 1:         
            diff = data_city[(data_city['cycle_1'] == 2019)]['p_female_city'].values - data_city[(data_city['cycle_1'] == 2014)]['p_female_city'].values
            data_merged.loc[(data_merged['city'] == city), 'p_female_city_diff'] = diff[0]

In [71]:
# generate differences in background shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'background', 'p_female_lr'])
data_panel = pd.DataFrame()
for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]
    
        for background in ('Links', 'Rechts'): 
            data_background = data_city[data_city['background'] == background]

            if len(data_background) > 1:
                diff = data_background[(data_background['cycle_1'] == 2019)]['p_female_lr'].values - data_background[(data_background['cycle_1'] == 2014)]['p_female_lr'].values
                data_merged.loc[(data_merged['city'] == city) & (data_merged['background'] == background), 'p_female_lr_diff'] = diff[0]

In [72]:
# generate differences in party-background shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'party', 'p_female_party_lr', 'background'])
data_panel = pd.DataFrame()

for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]
        
        for party in data_city['party'].unique():
            data_party = data_city[data_city['party'] == party]

            # drop units with only one party observation
            if len(data_party) > 1:
                for background in ('Links', 'Rechts'):
                    data_b = data_party[data_party['background'] == background]

                    # drop data w/ only one background info
                    if len(data_b) > 1:       

                        diff = data_b[(data_b['cycle_1'] == 2019)]['p_female_party_lr'].values - data_b[(data_b['cycle_1'] == 2014)]['p_female_party_lr'].values
                        data_merged.loc[(data_merged['party'] == party) & (data_merged['city'] == city), 'p_female_party_lr_diff'] = diff[0]

In [73]:
# import other files
data_official = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/controls.csv')
data_kreis = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/plz_kreis.csv', sep=';')

In [74]:
data_official.drop("Unnamed: 0",axis=1, inplace=True)
data_kreis.rename({"PLZ":'plz'}, axis=1, inplace=True)

In [75]:
data_merged = pd.merge(data_merged, data_kreis[['plz','Kreis']], on=['plz'], how='left', validate='m:1')
data_merged.rename({'Kreis':'county'}, axis=1, inplace=True)

In [76]:
# population control data
data_pop = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/controls.csv')
data_pop = data_pop[(data_pop['Land']==16)|(data_pop['Land']==14)|(data_pop['Land']==12)|(data_pop['Land']==8)]

In [77]:
data_pop.drop(['Unnamed: 0','Land','plz'], axis=1, inplace=True)

In [78]:
data_merged = pd.merge(data_merged, data_pop, on='city_id', how='left', validate='m:1')

In [81]:
data_merged[~data_merged['crime'].isna()]

,crime,city,law,date,background,suspects,party,state,plz,city_id,...,county,size,pop,pop_male,pop_female,density,longitude,latitude,travel_sig,degree_of_urban
0,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,Stuttgart,"207,33",635911.0,317794.0,318117.0,3067.0,"9,177734","48,775817",Region Stuttgart,dicht besiedelt
1,Verleumdung ohne Verleumdung ohne sexuelle Gru...,Stuttgart,STGB-187,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,Stuttgart,"207,33",635911.0,317794.0,318117.0,3067.0,"9,177734","48,775817",Region Stuttgart,dicht besiedelt
2,Volksverhetzung,Stuttgart,STGB-130,2019-08-12,Rechts,1.0,AfD,BW,70173.0,8111000.0,...,Stuttgart,"207,33",635911.0,317794.0,318117.0,3067.0,"9,177734","48,775817",Region Stuttgart,dicht besiedelt
3,Sachbeschadigung,Stuttgart,STGB-303,2019-08-26,Links,0.0,AfD,BW,70173.0,8111000.0,...,Stuttgart,"207,33",635911.0,317794.0,318117.0,3067.0,"9,177734","48,775817",Region Stuttgart,dicht besiedelt
4,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,70173.0,8111000.0,...,Stuttgart,"207,33",635911.0,317794.0,318117.0,3067.0,"9,177734","48,775817",Region Stuttgart,dicht besiedelt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21292,Diebstahl,Templin,STGB-242,2019-08-19,Links,0.0,AfD,BB,17268.0,12073572.0,...,Uckermark,"379,58",15728.0,7612.0,8116.0,41.0,"13,499106","53,120644",Uckermark,mittlere Besiedlungsdichte
21337,Sachbeschadigung,Oberuckersee,STGB-303,2019-08-30,Nicht zuzuordnen,0.0,CDU,BB,17291.0,12073430.0,...,Uckermark,"84,96",1629.0,836.0,793.0,19.0,"13,840199","53,217306",Uckermark,gering besiedelt
21338,Sachbeschadigung,Oberuckersee,STGB-303,2019-08-30,Nicht zuzuordnen,0.0,CDU,BB,17291.0,12073430.0,...,Uckermark,"84,96",1629.0,836.0,793.0,19.0,"13,840199","53,217306",Uckermark,gering besiedelt
21370,Verwenden von Kennzeichen verfassungsfeindlich...,Grosskmehlen,STGB-86A,2019-08-02,Rechts,0.0,CDU,BB,1990.0,12066104.0,...,Oberspreewald-Lausitz,"13,88",1058.0,536.0,522.0,76.0,"13,72453","51,378507",Lausitzer Seenland,gering besiedelt


In [82]:
data_merged.to_csv(r'C:\Users\mariu\Documents\pol_viol\pol_viol\data_election_crime_merged.csv')